<a href="https://colab.research.google.com/github/JoanneT8/FEN-Chess-Identifier-APS360-Summer-2021/blob/main/FEN_Generator_APS360_Summer_2021_Main_Model_using_YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Google Colab Shared Link
Baseline Model Link: https://colab.research.google.com/drive/1hBLG7VLoNU6S6vvo8Ff9rpU0ix5F1Wwj?usp=sharing

Main Model Link: https://colab.research.google.com/drive/1gLG2Td7MzgWcH2JUzr1E3Rst3kRTovYm?usp=sharing 

# Creating the Main Model - Utilizing YOLOv5

- Using Pre-trained model (YOLOv5x) from https://github.com/ultralytics/yolov5

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Set-up

In [2]:
# setting up dependencies needed to use YOLOv5
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 9310, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 9310 (delta 26), reused 43 (delta 22), pack-reused 9260
Receiving objects: 100% (9310/9310), 9.64 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (6469/6469), done.


In [3]:
cd yolov5

/content/yolov5


In [4]:
pip install -qr requirements.txt # installing dependencies needed for YOLOv5

     |████████████████████████████████| 3.0 MB 5.1 MB/s 
     |████████████████████████████████| 636 kB 40.8 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 7.3 MB/s 
     |████████████████████████████████| 170 kB 67.8 MB/s 
     |████████████████████████████████| 138 kB 72.3 MB/s 
     |████████████████████████████████| 97 kB 9.0 MB/s 
     |████████████████████████████████| 133 kB 49.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6502 sha256=2a40165a50e8ee3c468d574f29c6217d56666ef566c84f8213b0ddd9d82ac3db
  Stored in directory: /root/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=c335ab8de1aaa58c6589cae07f9cfeef0f330c48fa258dcc730b367adfbc60e1
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools
  Attempting uninstall: urlli

In [ ]:
# not sure what -qqq is, hopefully it's something related to saving models? I saw this on a website somewhere: https://wandb.ai/lavanyashukla/save_and_restore/reports/Saving-and-Restoring-Models-with-W-B--Vmlldzo3MDQ3Mw
!pip install wandb -qq

In [ ]:
''' 
Wandb (Weights and Biases) is a service that directly connects to the YOLOv5 model 
that will help track training curves, F1 score, and confusion matrices
'''
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
Aborted!
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 623, in _exitfunc
    @classmethod
KeyboardInterrupt
Exception ignored in: <finalize object at 0x7f0c5f993bb0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
TypeError: 'NoneType' object is not callable
Exception ignored in: <finalize object at 0x7f0c5f9935b0; dead>
Traceback (most recent call last):
  File "/usr/lib/python3.7/weakref.py", line 572, in __call__
  File "/usr/lib/python3.7/tempfile.py", line 936, in _cleanup
TypeError: 'NoneType' object is not callable


In [5]:
import torch
from IPython.display import Image, clear_output  # to display images

In [6]:
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla K80)


In [ ]:
# split data into train, validation, and test sets in a 70:15:15 ratio (ratio can be changed but we'll stick with this for now)
# annotated images: 120 (Kevin) + 45 (Morgan) + 68 (John) + 120 (Joanne) = 343 images
# train = 240 
# validation = 52 + 51

# NOTE: Make sure the YOLO annotation files are in the same directory as the images themselves

In [ ]:
# Modify YAML file to specify path to train and validation folder, and info about number of classes and the names of those classes
# hmm unsure of this part, the YAML file only specifies the number of classes (nc)
"""
example from:  https://michaelohanu.medium.com/yolov5-tutorial-75207a19a3aa 

train: /Users/macbook/Desktop/antenna/yolov5/data/train.txt
val: /Users/macbook/Desktop/antenna/yolov5/data/val.txt

# number of classes
nc: 1
# class names
names: ['antenna'] 

"""

"\nexample from:  https://michaelohanu.medium.com/yolov5-tutorial-75207a19a3aa \n\ntrain: /Users/macbook/Desktop/antenna/yolov5/data/train.txt\nval: /Users/macbook/Desktop/antenna/yolov5/data/val.txt\n\n# number of classes\nnc: 1\n# class names\nnames: ['antenna'] \n\n"

In [7]:
cd /content/yolov5

/content/yolov5


In [8]:
# downloading our .yaml file for our chess dataset -> it specifies the train and validation folder locations and the classes
!gdown --id 1W9J7rGbrKBwyzYtSctip9irltiwQ8rYi

Downloading...
From: https://drive.google.com/uc?id=1W9J7rGbrKBwyzYtSctip9irltiwQ8rYi
To: /content/yolov5/FEN.yaml
100% 362/362 [00:00<00:00, 683kB/s]


## Training


Combinations training log:
b = Batch, e = epoch

- yolov5x, b = 15, e = 100 (Aug 3, 12:05) Time taken: 53 min 55 sec 
    - validation loss is starting to level out (don't really need more epochs?), let's try changing the batch size next
- yolov5x, b = 15, e = 100 (Aug 3, 12:05) Using ADAM OPTIMIZER Time taken:  ~ 1 hour
- yolov5x, b = 30, e = 100 (Aug 3, to be done) Time taken: 

In [ ]:
# ok this didn't do what I thought it would do... i thought it would help save the model but i don't think so
# wandb.init(project="YOLOv5") - Joanne

In [ ]:
WANDB_PROJECT = 'YOLOv5'

In [9]:
import tensorflow as tf
import datetime, os

In [ ]:
!python train.py --img 200 --batch 50 --epochs 200 \
  --data /content/yolov5/FEN.yaml --weights yolov5x.pt \
  --name yolov5x_b50_ep200_run_train282_val61_garbage --cache --project /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5

train: weights=yolov5x.pt, cfg=, data=/content/yolov5/FEN.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=200, batch_size=50, imgsz=200, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5, entity=None, name=yolov5x_b50_ep200_run_train282_val61_garbage, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-358-g3e7c59a torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0

In [10]:
!python train.py --img 200 --batch 15 --epochs 100 \
  --data /content/yolov5/FEN.yaml --weights yolov5x.pt \
  --name yolov5x_b15_ep100_run_train282_val61_copy --cache --project /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5

100% 755k/755k [00:00<00:00, 18.5MB/s]
train: weights=yolov5x.pt, cfg=, data=/content/yolov5/FEN.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=15, imgsz=200, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5, entity=None, name=yolov5x_b15_ep100_run_train282_val61_copy, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=30
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-405-gfad57c2 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=

## Final Model

Fine Tuned Hyperparameters: 
- SGD optimizer (Adam optimizer resulted in very jaggy/inconsistent/unpredictable training curves)
- Batch size = 15
- Epoch = 100
- YOLOv5 model adjusts learning rate as the model trains

Dataset:
- 282 train images
- 61 validation images
- 61 test images

**NOTE:** Did not keep all the models that we've fine tuned. Final model that gives best accuracy is located here:  /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt



**Results logged here:** 

https://wandb.ai/aps360-group3-2021/YOLOv5/reports/FEN-Generator-APS360-Summer-2021-Training-Report-Results--Vmlldzo5ODUzMjY?accessToken=wzxixfsd9pmub4cqhmlsfahkszqpaxerv1xeu3rdfhnbkoi0g5ionlw1zqnjko72

# Testing

## Run Model through Test Images

- Trying a bunch of different command combinations:



In [ ]:
""" 
run the detect.py script that comes with YOLOv5 and if you give it a folder of images or an image itself, specify the model you want to run after 

Additional actions you can specify and modify (from reading the code in the YOLOv5 GitHub):
--weights -> (best.pt is a file where the script keeps track of your best model/epoch and runs detect.py based on that -> make sure to change the file location to which model you want to use!)

--img -> specifies the size of our images (which is 200x200px)
--conf -> is the confidence level of the prediction, only shows the predictions above this confidence level numbers
--source -> is where the image you want the model to analyze is located
--save-crop -> Will crop and save (as JPEG files) the objects the model has detected with bounding boxes
--save-txt -> saves the bounding box coordinates, class predicited, and the confidence of the prediction into txt files

"""
!python detect.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
  --img 200 --conf 0.3 --line-thickness 1 --save-txt --save-conf --save-crop \
  --source /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated \
  --project /content/drive/MyDrive/APS360_Project_Group_3/test_results

detect: weights=['/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt'], source=/content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated, imgsz=200, conf_thres=0.3, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/APS360_Project_Group_3/test_results, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-350-gf409d8e torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs
image 1/61 /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated/1B1B3R-8-1b2k3-8-2n5-1rK5-b7-8.jpeg: 224x224 2 b_bs, 2 b_ws, 1 r_b, 1 r_w, 1 n_b, 1 k_b, 1 k_w, Done. (0.041s)
image 2/61 /content/drive/MyDrive/A

In [ ]:
!python export.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt

export: weights=/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt, img_size=[640, 640], batch_size=1, device=cpu, include=['torchscript', 'onnx', 'coreml'], half=False, inplace=False, train=False, optimize=False, dynamic=False, simplify=False, opset=12
YOLOv5 🚀 v5.0-350-gf409d8e torch 1.9.0+cu102 CPU

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs

PyTorch: starting from /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt (175.2 MB)

TorchScript: starting export with torch 1.9.0+cu102...
/content/yolov5/yolov5/yolov5/yolov5/models/yolo.py:57: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] !=

In [ ]:
!python detect.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
--data /content/yolov5/FEN.yaml --iou-thres 0.5 \
 --task test --source

usage: detect.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE]
                 [--imgsz IMGSZ] [--conf-thres CONF_THRES]
                 [--iou-thres IOU_THRES] [--max-det MAX_DET] [--device DEVICE]
                 [--view-img] [--save-txt] [--save-conf] [--save-crop]
                 [--nosave] [--classes CLASSES [CLASSES ...]] [--agnostic-nms]
                 [--augment] [--visualize] [--update] [--project PROJECT]
                 [--name NAME] [--exist-ok] [--line-thickness LINE_THICKNESS]
                 [--hide-labels] [--hide-conf] [--half]
detect.py: error: argument --source: expected one argument


In [ ]:
!python detect.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
  --save-txt --save-conf --iou-thres 0.5 \
  --source /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated \
  --project /content/drive/MyDrive/APS360_Project_Group_3/test_results

detect: weights=['/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt'], source=/content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated, imgsz=640, conf_thres=0.25, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/APS360_Project_Group_3/test_results, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-350-gf409d8e torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs
image 1/61 /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated/1B1B3R-8-1b2k3-8-2n5-1rK5-b7-8.jpeg: 640x640 2 b_bs, 2 b_ws, 1 r_b, 1 r_w, 1 n_b, 1 k_b, 1 k_w, Done. (0.069s)
image 2/61 /content/drive/MyDrive/

In [ ]:
!python detect.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
  --img 200 --iou-thres 0.5 --conf 0.3 --line-thickness 1 --save-txt --save-conf --save-crop \
  --source /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated/1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR.jpeg \
  --project /content/drive/MyDrive/APS360_Project_Group_3/test_results/1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR.jpeg


detect: weights=['/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt'], source=/content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated/1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR.jpeg, imgsz=200, conf_thres=0.3, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/APS360_Project_Group_3/test_results/1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR.jpeg, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs
image 1/1 /content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated/1B1K4-3N1k1P-6Q1-3r1n2-8-3p1NNN-8-3b2rR.jpeg: 224x224 1

In [ ]:

!python train.py --img 200 --batch 50 --epochs 200 \
  --data /content/yolov5/FEN.yaml --weights yolov5x.pt \
  --name yolov5x_b50_ep200_run_train282_val61 --cache --project /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5

In [ ]:
# trying to use val.py
!python val.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
  --img 200 --task test --data /content/yolov5/FEN.yaml \
  --save-txt --save-conf --iou-thres 0.5 --conf-thres 0.9 \
  --project /content/drive/MyDrive/APS360_Project_Group_3/test_results/conf_90

val: data=/content/yolov5/FEN.yaml, weights=['/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt'], batch_size=32, imgsz=200, conf_thres=0.9, iou_thres=0.5, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=True, save_hybrid=False, save_conf=True, save_json=False, project=/content/drive/MyDrive/APS360_Project_Group_3/test_results/conf_90, name=exp, exist_ok=False, half=False
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs
test: Scanning '/content/drive/MyDrive/APS360_Project_Group_3/dataset/annotated/test_annotated.cache' images and labels... 61 found, 0 missing, 0 empty, 0 corrupted: 100% 61/61 [00:00<00:00, 582807.62it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 2/2 [00:01<00:00,  1.89it/s]
                 all         61          0  

## Testing Completely New Data from Google Images

Steps to test new images on our trained model:
1. Find image you want to convert into FEN notation (make sure it is a screenshot of only the chess board with chess pieces on it for best results)
2. Save it onto Google Drive (remember where you saved it! You will need the path to the image)
3. Change the --source to the image path from Step 2
4. Make sure you are using the best model (in our case, the best model is called yolov5x_b15_ep100_run_train282_val61/weights/best.pt)
  - best.pt saves the best run out of the 100 epochs we ran our model through
5. Use --project to specify where you want the model to output the result (will output the image inputted with it's bounding box and class predictions

In [ ]:
# testing a random images from google
!python detect.py --weights /content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt \
  --img 200 --iou-thres 0.5 --conf 0.5 --line-thickness 1 --save-txt --save-conf --save-crop \
  --source /content/drive/MyDrive/APS360_Project_Group_3/dataset/random_chess_screenshots/aps2.png \
  --project /content/drive/MyDrive/APS360_Project_Group_3/test_results/random_screenshot/aps2

detect: weights=['/content/drive/MyDrive/APS360_Project_Group_3/YOLOv5/yolov5x_b15_ep100_run_train282_val61/weights/best.pt'], source=/content/drive/MyDrive/APS360_Project_Group_3/dataset/random_chess_screenshots/aps2.png, imgsz=200, conf_thres=0.5, iou_thres=0.5, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/APS360_Project_Group_3/test_results/random_screenshot/aps2, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-351-ge96c74b torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 476 layers, 87272713 parameters, 0 gradients, 217.3 GFLOPs
image 1/1 /content/drive/MyDrive/APS360_Project_Group_3/dataset/random_chess_screenshots/aps2.png: 224x224 7 p_bs, 7 p_ws, 1 b_b, 1 b_w, 2 r_bs, 2 r_ws, 1 n_b, 1 n_w, 1 k_b, 1 k_w, 1 q_b, 1 q_w, Done. (0.0

# Extracting bounding box position of prediction to convert to FEN



- We read the text file that the model outputs and extract the bounding box location of each object detected
- We map each detected object to it's corresponding piece and position on the board


In [ ]:
def pieceAndName(string):
  if(string == "Rook"):
    return "R"
  elif(string == "Queen"):
    return "Q"
  elif(string == "Pawn"):
    return "P"
  elif(string == "Knight"):
    return "N"
  elif(string == "King"):
    return "K"
  elif(string == "Bishop"):
    return "B"
  elif(string == "rook"):
    return "r"
  elif(string == "queen"):
    return "q"
  elif(string == "pawn"):
    return "p"
  elif(string == "knight"):
    return "n"
  elif(string == "king"):
    return "k"
  elif(string == "bishop"):
    return "b"

In [ ]:


#print("piece + position on the board: ")
def get_fen(f):
  fen = {}
  for x in f:
    ind = x.split()
    v_axis = 224*float(ind[1])
    h_axis = 224*float(ind[2])
    sep = 224/8
    piece=''
    p1=''
    p2=''
    if ind[0]=='0':
      piece='pawn'
    elif ind[0]=='1':
      piece='Pawn'
    elif ind[0]=='2':
      piece='bishop'
    elif ind[0]=='3':
      piece='Bishop'
    elif ind[0]=='4':
      piece='rook'
    elif ind[0]=='5':
      piece='Rook'
    elif ind[0]=='6':
      piece='knight'
    elif ind[0]=='7':
      piece='Knight'
    elif ind[0]=='8':
      piece='king'
    elif ind[0]=='9':
      piece='King'
    elif ind[0]=='10':
      piece='queen'
    elif ind[0]=='11':
      piece='Queen'
    if v_axis >0 and v_axis<=sep:
      p1='A'
    if v_axis >sep and v_axis<=2*sep:
      p1='B'
    if v_axis >2*sep and v_axis<=3*sep:
      p1='C'
    if v_axis >3*sep and v_axis<=4*sep:
      p1='D'
    if v_axis >4*sep and v_axis<=5*sep:
      p1='E'
    if v_axis >5*sep and v_axis<=6*sep:
      p1='F'
    if v_axis >6*sep and v_axis<=7*sep:
      p1='G'
    if v_axis >7*sep and v_axis<=8*sep:
      p1='H'

    if h_axis >0 and h_axis<=sep:
      p2='8'
    if h_axis >sep and h_axis<=2*sep:
      p2='7'
    if h_axis >2*sep and h_axis<=3*sep:
      p2='6'
    if h_axis >3*sep and h_axis<=4*sep:
      p2='5'
    if h_axis >4*sep and h_axis<=5*sep:
      p2='4'
    if h_axis >5*sep and h_axis<=6*sep:
      p2='3'
    if h_axis >6*sep and h_axis<=7*sep:
      p2='2'
    if h_axis >7*sep and h_axis<=8*sep:
      p2='1'
    fen.setdefault(p2, []).append(piece)
    fen.setdefault(p2, []).append(p1)
    
    
    #print('{} --> {}{}'.format(piece,p1,p2))


  c=8
  output=''
  for x in range(8):
    y=c-x
    count =0
    if str(y) in fen:
      if'A' in fen.setdefault(str(y), {}):
        output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('A')-1])
        count = 1
      if'B' in fen.setdefault(str(y), {}):
        if count== 0: 
          output += '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('B')-1])
        if count== 1:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('B')-1])
        count =2
      if'C' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('C')-1])
        if count == 1:
          output += '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('C')-1])
        if count ==2:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('C')-1])
        count = 3
      if'D' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '3'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('D')-1])
        if count == 1:
          output += '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('D')-1])
        if count ==2:
          output += '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('D')-1])
        if count ==3:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('D')-1])
        count = 4
      if 'E' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '4'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('E')-1])
        if count == 1:
          output += '3'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('E')-1])
        if count ==2:
          output += '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('E')-1])
        if count ==3:
          output += '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('E')-1])
        if count ==4:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('E')-1])
        count = 5
      if'F' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '5'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        if count == 1:
          output += '4'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        if count ==2:
          output += '3'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        if count ==3:
          output += '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        if count ==4:
          output+= '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        if count ==5:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('F')-1])
        count = 6
      if'G' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '6'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count == 1:
          output += '5'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count ==2:
          output += '4'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count ==3:
          output += '3'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count ==4:
          output+= '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count ==5:
          output+= '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        if count ==6:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('G')-1])
        count = 7
      if'H' in fen.setdefault(str(y), {}):
        if count ==0:
          output += '7'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count == 1:
          output += '6'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==2:
          output += '5'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==3:
          output += '4'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==4:
          output+= '3'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==5:
          output+= '2'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==6:
          output+= '1'
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        if count ==7:
          output += pieceAndName(fen.setdefault(str(y), {})[fen.setdefault(str(y), {}).index('H')-1])
        count =8
      if count ==7:
        output+= '1'
      if count ==6:
        output+= '2'
      if count ==5:
        output+= '3'
      if count ==4:
        output+= '4'
      if count ==3:
        output+= '5'
      if count ==2:
        output+= '6'
      if count ==1:
        output+= '7'
    else:
      output+= '8'
    output+= '-'
  return output[:-1]


### Computing the accuracy of our model predicition to FEN notation conversion

In [ ]:
import os
path = '/content/drive/MyDrive/APS360_Project_Group_3/test_results/conf_50/exp/labels'
correct = 0
total = 0
for i in os.listdir(path):
 
  ground = i[:-4]
  
  f = open(os.path.join(path, i), "r")
  test = get_fen(f)

  if test==ground:
    correct +=1
  else:
    print (test)
    print (ground)
  total += 1
  

print ("The fen accuracy is : " + str (correct/total))

1b1K4-R5k1-4r2N-8-8-8-8-K7
1b1K4-R5k1-4r2N-8-8-8-8-Q7
1b1NN3-3N3p-3K4-8-1K4b1-2R5-5r2-5N1B
1b1NN3-3N3p-3k4-8-1K4b1-2R5-5r2-5N1B
1B1b4-3q4-n6k-3b3R-Q7-1K3p2-1n6-1N3QRb
1b1b4-3q4-n6k-3b3R-Q7-1K3p2-1n6-1N3QRb
1B1B4-4r3-2r2p2-K7-8-3b3r-2Q4P-R2k4
1B1B4-4R3-2R2p2-K7-8-3b3r-2Q4P-R2k4
1B1K4-1p5N-7p-1qp5-n1P5-8-6k1-B7
1B1K4-1p5N-7p-1qp5-n1P5-8-6k1-b7
1B1n4-2p1b3-p1K1PBr1-2b4k-6B1-8-8-BN6
1B1n4-2p1b3-P1K1PBR1-2b4k-6B1-8-8-BN6
The fen accuracy is : 0.9016393442622951


### Get Final Output in FEN

In [ ]:
i= "/content/drive/MyDrive/APS360_Project_Group_3/test_results/random_screenshot/aps2/exp/labels/aps2.txt"
f = open(i, "r")
print (get_fen(f).replace("-","/"))

3r1rk1/1pR1bpp1/3p3p/pP2p3/P3Pnq1/BP1Q1NP1/5P1P/3R2K1


# Testing model with Random Screenshots
- Taking output from model, reading the text file, then converting directly to FEN using the get_fen() function

In [ ]:
i= "/content/drive/MyDrive/APS360_Project_Group_3/test_results/random_screenshot/wikihow-Win-Chess-Openings_-Playing-Black-Step-14.jpg/exp/labels/wikihow-Win-Chess-Openings_-Playing-Black-Step-14.txt"
f = open(i, "r")
print (get_fen(f).replace("-","/"))

1nkpbb2/1K1k1kk1/4K3/2BN4/1KKK1K2/1KNKKKK1/1PNNKNP1/8


In [ ]:
i= "/content/drive/MyDrive/APS360_Project_Group_3/test_results/random_screenshot/basic-principles-of-chess-openings-611601-control-the-center.png/exp/labels/basic-principles-of-chess-openings-611601-control-the-center.txt"
f = open(i, "r")
print (get_fen(f).replace("-","/"))

r1bkkb1r/1pppppp1/n6n/p6p/3RP3/2N2N2/PPR2PRP/R1BQKB1R
